# Welcome!

Jupyter repo for this tutorial: https://github.com/reuven/2024-pycon-decorators/

# Agenda

1. What are decorators?
2. Writing your first decorator
3. Outer function storage
4. Inputs and outputs
5. Decorators that take arguments
6. Nested decorators
7. Decorating classes
8. Writing decorators as classes

# What are decorators?

Let's assume that we have two basic functions.

In [2]:
def a():
    return f'A!\n'

def b():
    return f'B!\n'

print(a())
print(b())

A!

B!



In [3]:
lines = '-' * 60 + '\n'

def a():
    return f'{lines}A!\n{lines}'

def b():
    return f'{lines}B!\n{lines}'

print(a())
print(b())

------------------------------------------------------------
A!
------------------------------------------------------------

------------------------------------------------------------
B!
------------------------------------------------------------



# Don't Repeat Yourself (DRY) rule of programming

Right now, we have a technique that will require us to include `lines` in every function we write!

How can I include `lines` without having to include that variable explicitly in every function?

In [4]:
# option 1: Write a new function that takes a function as an argument

lines = '-' * 40 + '\n'

def with_lines(func):
    return f'{lines}{func()}{lines}'

def a():
    return f'A!\n'

def b():
    return f'B!\n'

print(with_lines(a))
print(with_lines(b))

----------------------------------------
A!
----------------------------------------

----------------------------------------
B!
----------------------------------------



In [5]:
# option 2: turn with_lines into a function that returns a function
# that is: We're going to create a closure!

lines = '-' * 40 + '\n'

def with_lines(func):
    def wrapper():
        return f'{lines}{func()}{lines}'
    return wrapper

def a():
    return f'A!\n'
with_lines_a = with_lines(a)

def b():
    return f'B!\n'

with_lines_b = with_lines(b)

print(with_lines_a())
print(with_lines_b())

----------------------------------------
A!
----------------------------------------

----------------------------------------
B!
----------------------------------------



In [7]:
# option 3: don't assign to with_lines_a and with_lines_b
# rather, assign to a and b

lines = '-' * 40 + '\n'

def with_lines(func):
    def wrapper():
        return f'{lines}{func()}{lines}'
    return wrapper

def a():
    return f'A!\n'
a = with_lines(a)

def b():
    return f'B!\n'
b = with_lines(b)

print(a())
print(b())

----------------------------------------
A!
----------------------------------------

----------------------------------------
B!
----------------------------------------



In [8]:
x = 5
x = 7
print(x)

7


In [9]:
# option 4: Use Python decorator syntax

lines = '-' * 40 + '\n'

def with_lines(func):
    def wrapper():
        return f'{lines}{func()}{lines}'
    return wrapper

@with_lines    # this is 100% equivalent to what's in line 13 -- decorator syntax
def a():
    return f'A!\n'
# a = with_lines(a)

@with_lines
def b():
    return f'B!\n'
# b = with_lines(b)

print(a())
print(b())

----------------------------------------
A!
----------------------------------------

----------------------------------------
B!
----------------------------------------



# Who needs this? 

1. if I have many functions that need to do the same thing, I can extract that functionality into a decorator, and thus DRY up my code.
2. Timing my functions
3. Security of my functions
4. Setting values before/after (setup/teardown)
5. Logging
6. Checking/filtering inputs
7. Filtering outputs

In [11]:
# let's decorate another function!

@with_lines
def add(x, y):
    return f'{x + y}\n'

print(add(3, 5))

TypeError: with_lines.<locals>.wrapper() takes 0 positional arguments but 2 were given

In [13]:

def with_lines(func):
    def wrapper(*args):   # take any number of positional arguments, create a tuple, args
        return f'{lines}{func(*args)}{lines}'  # turn args into arguments to func()
    return wrapper

@with_lines
def add(x, y):
    return f'{x + y}\n'

print(add(3, 5))

----------------------------------------
8
----------------------------------------



# How to write a decorator

1. The outer function, the decorator, takes one argument, a function (`func`) -- this is the function that will be decorated. This outer function is invoked once, when we define the decorated function.
2. The inner function, typically called `wra